In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count
import pyspark.pandas as pd
from pyspark.sql import functions as F
import pyspark.sql.types as types

In [51]:
spark = SparkSession.builder \
        .appName("Area surfaces with rooms and property type") \
        .config("spark.jars", "../jdbc/mssql-jdbc-12.6.1.jre8.jar") \
        .getOrCreate()

In [52]:
server_name = "mssql"
port = "1433"
database_name = "Data"
url = f"jdbc:sqlserver://{server_name}:{port};databaseName={database_name}"

table_name = "ARProperties"
username = "SA"
password = "YourStrongPassword123"

df = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password) \
        .option("encrypt", "false") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()

In [53]:
df = df.select(df.property_type, df.surface_total, df.surface_covered, df.rooms, df.bedrooms, df.bathrooms)

In [54]:
df = df.filter(~col("property_type").isin("Terrain"))

In [61]:
def fix_rooms(rooms, bathrooms, bedrooms):
    fix = rooms
    if not(None in rooms, bathrooms, bedrooms):
        if rooms < bathrooms+bedrooms:
            fix = bathrooms+bedrooms
    
    return fix

def fix_bathrooms(bathrooms):
    if bathrooms == 0 or bathrooms == None:
        return 1

def fix_bedrooms(bedrooms):
    if bedrooms == None:
        return 0

fix_bathrooms = F.udf(fix_bathrooms, types.IntegerType())
fix_rooms = F.udf(fix_rooms, types.IntegerType())

In [57]:
df = df.withColumns({
    "rooms":col("rooms").cast("integer"),
    "bathrooms":col("bathrooms").cast("integer"),
    "bedrooms":col("bedrooms").cast("integer")
})

In [62]:
df = df.withColumn("bathrooms", fix_bathrooms(col("bathrooms")))
df = df.withColumn("rooms", fix_rooms(col("rooms"), col("bathrooms"), col("bedrooms")))

In [63]:
df.show(20)

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_1162/3337372527.py", line 3, in fix_rooms
TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'


In [13]:
df.groupBy("property_type") \
           .agg(count(when(col("surface_total").isNull(), True)).alias("null_count"),
                count(when(col("surface_total").isNotNull(), True)).alias("not_null_count")).show()

+-------------------+----------+--------------+
|      property_type|null_count|not_null_count|
+-------------------+----------+--------------+
|          Apartment|     45178|         44671|
|      Village House|       243|           198|
|             Office|      3078|          3815|
|Commercial Premises|      3673|          4363|
|              Other|     62320|          1573|
|            Terrain|     12078|          7540|
|             Garage|      1539|           769|
|          Warehouse|       709|           784|
|              House|     29154|         20093|
|Horizontal Property|      4778|          3444|
+-------------------+----------+--------------+

